In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY = os.getenv('OPENAI_API_KEY')

In [5]:
llm = ChatOpenAI(openai_api_key = KEY, temperature=0.3, model='gpt-3.5-turbo')

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [6]:
RESPONSE_JSON = {
        "1": {
            "mcq": "multiple choice question",
            "options":{
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here",
            },
            "correct": "correct answer",
        },

       "2": {
            "mcq": "multiple choice question",
            "options":{
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here",
            },
            "correct": "correct answer",
        },
}

In [7]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
TEMPLATE = '''
        Text: {text}
        You are an expert MCQ maker. Given the above text, it is your job to \
        create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
        Make sure the questions are not repeated and check all the questions to be conforming the text as well.
        Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
        Ensure to make {number} MCQs
        ### RESPONSE_JSON
        {response_json}

        '''

In [9]:
quiz_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_prompt, output_key = "quiz", verbose =True)

In [11]:
TEMPLATE2="""
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at par with the cognitive and analytical abilities of the students, \
update the quiz questions which need to be changed and change the tone such that it perfectly fits the student's ability
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [13]:
quiz_evaluation_chain = LLMChain(llm = llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
quiz_sequence = SequentialChain(chains=[quiz_chain,quiz_evaluation_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                               output_variables=["quiz","review"], verbose=True)

In [16]:
file_path = r'D:\languages\python\projects\GenerativeAI_projects\MCQ_generator_openAI_and_langchain\data.txt'

In [17]:
with open(file_path,'r') as file:
    text = file.read()
    

In [18]:
text

"Deep learning (DL) is a subset of machine learning (ML) that deals with algorithms inspired by the structure and function of the human brain, known as artificial neural networks. These networks consist of interconnected layers of nodes (neurons) that process information in a hierarchical manner.\n\nHere's how DL differs from traditional machine learning:\n\n1. **Hierarchy of Representation:** DL algorithms automatically learn to represent data in multiple layers of increasing complexity, allowing the system to discover intricate patterns within the data. This hierarchical representation is often more abstract and is one of the significant differences from traditional ML.\n\n2. **Feature Extraction:** In traditional ML, domain experts often handcraft the features used for training models. In DL, these features are learned automatically from the raw data, reducing the need for manual feature engineering.\n\n3. **Scale and Performance:** DL algorithms can handle vast amounts of data effi

In [19]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER =3
SUBJECT = 'deeplearning'
TONE ='simple'

In [25]:
with get_openai_callback() as cb:
    response = quiz_sequence(
        {
        "text": text,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        Text: Deep learning (DL) is a subset of machine learning (ML) that deals with algorithms inspired by the structure and function of the human brain, known as artificial neural networks. These networks consist of interconnected layers of nodes (neurons) that process information in a hierarchical manner.

Here's how DL differs from traditional machine learning:

1. **Hierarchy of Representation:** DL algorithms automatically learn to represent data in multiple layers of increasing complexity, allowing the system to discover intricate patterns within the data. This hierarchical representation is often more abstract and is one of the significant differences from traditional ML.

2. **Feature Extraction:** In traditional ML, domain experts often handcraft the features used for training models. In DL, these features are learned automatically from the raw data, reducing the need for m

In [39]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1328
Prompt Tokens: 1021
Completion Tokens: 307
Total Cost: 0.0021455


In [40]:
quiz = response.get("quiz")

In [47]:
type(quiz)

str

In [48]:
quiz = json.loads(quiz)

In [49]:
import pandas as pd

In [52]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = "|".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items() ]
        
    )
    correct = value["correct"]
    quiz_table_data. append({"MCQ": mcq,"Choices": options, "Correct": correct})

In [53]:
quiz = pd.DataFrame(quiz_table_data)

In [56]:
print(quiz["Choices"])

0    a: Deep learning models require less computati...
1    a: Tasks with small datasets|b: Tasks with sim...
2    a: When interpretability is not important|b: W...
Name: Choices, dtype: object


In [57]:
quiz


,MCQ,Choices,Correct
0,What is one of the significant differences bet...,a: Deep learning models require less computati...,b
1,Which type of tasks have seen substantial impr...,a: Tasks with small datasets|b: Tasks with sim...,c
2,When might traditional machine learning method...,a: When interpretability is not important|b: W...,a


In [59]:
quiz.to_csv("Quiz_table.csv",index=False)